<a href="https://colab.research.google.com/github/frizuma3/21013159_DataAnalytics/blob/main/Linear%20Regression%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINEAR REGRESSION ANALYSIS

In this Assignment2 we will train and test a linear regressor.
We will use the data generated from the DAOTW Collision Assignment.

This assignment is carried out in Python-Kernel using Google_Colab.

In [ ]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/frizuma3/21013159_DataAnalytics/refs/heads/main/linearregressiondata.csv', index_col=0, )

In [ ]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

   day  temp  dewp  NUM_COLLISIONS
1    1  83.6  63.0        0.520270
2    3  80.3  54.1        0.578829
3    4  79.8  56.7        0.804054
4    5  81.8  65.6        0.281532
5    6  86.7  64.3        0.639640
6    7  81.9  62.3        0.745495


In [ ]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_COLLISIONS = 1.0

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.17.0


# Day and NUM_COLLISIONS

In [ ]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration, in this case, I have gone for day, temp, dewp
df_input_data_day = [df["day"], df["NUM_COLLISIONS"]]
# create headers for our new dataframe. These should correlate with the above.
df_input_headers_day = ["day", "NUM_COLLISIONS"]
# create a final dataframe using our new dataframe and headers.
df_input_day = pd.concat(df_input_data_day, axis=1, keys=df_input_headers_day)

In [ ]:
# construct a training set for runnign through the model and a test set, we do this by using sample with 0.8 for an 80% training set and 20% for test.
training_set_day = df_input_day.sample(frac=0.8, random_state=0)
test_set_day = df_input_day.drop(training_set_day.index)

In [ ]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features_day = training_set_day.copy()
test_features_day = test_set_day.copy()

training_labels_day = training_features_day.pop('NUM_COLLISIONS')
test_labels_day = test_features_day.pop('NUM_COLLISIONS')

In [ ]:
# Here I have put in a scale factor and divided by it. In this dataset, I had already normalised and thus it is 1. However, 600000 is what would make sense based on the data here and we can use this later when testing our model..
training_labels_day = training_labels_day/SCALE_NUM_COLLISIONS
test_labels_day = test_labels_day/SCALE_NUM_COLLISIONS

In [ ]:
print(training_features_day)

      day
942     5
623     2
902     7
2050    7
575     1
...   ...
2285    4
1191    1
340     5
47      6
754     6

[2174 rows x 1 columns]


In [ ]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser_day = tf.keras.layers.Normalization(input_shape=[1,], axis=None) # tf.keras.layers.Normalization(axis=-1)
normaliser_day.adapt(np.array(training_features_day))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# I have decided to call the model, model_1. We add our normaliser and we are expecting a single output.
model_0 = tf.keras.Sequential([
    normaliser_day,
    layers.Dense(units=1)
])

In [ ]:
model_0.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
# now we are going to fit the model where we require the training features and labels. We will run it 100 times i.e. epochs and we have applied a further 20% validation split.

%%time
history = model_0.fit(
    training_features_day,
    training_labels_day,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 12.2 s, sys: 698 ms, total: 12.9 s
Wall time: 14 s


In [ ]:
# Now, we will evaluate our model using the test features and labels.
mean_absolute_error_model_0 = model_0.evaluate(
    test_features_day,
    test_labels_day, verbose=0)

In [ ]:
# The mean absolute error of the model can be printed out. Remember, we want to minimise this. Perhaps a model with just day and NUM_COLLISIONS would be better. It will also vary on each training run due to randomisation.
print(mean_absolute_error_model_0)

0.1443050056695938


In [ ]:
# we create a custom dataframe with 3 values per feature.
input_model_0 = pd.DataFrame.from_dict(data =
				{
            'day' : [1,2,3,4,5,6,7],
        })

In [ ]:
linear_predictions_model_0 = model_0.predict(input_model_0)*SCALE_NUM_COLLISIONS # essentially 10000 in this instance would give back realistic numbers based on the NUM_COLLISIONS data
print(linear_predictions_model_0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.48049745]
 [0.49589744]
 [0.5112974 ]
 [0.5266974 ]
 [0.5420974 ]
 [0.5574974 ]
 [0.5728974 ]]


# Day, Temp, DewPoint, NUM_COLLISIONS


In [ ]:
# create a dataframe with the inputs and the output at the end using the imported dataframe. This can be replicated for any configuration, in this case, I have gone for day, temp, dewp
df_input_data = [df["day"], df["temp"], df["dewp"], df["NUM_COLLISIONS"]]
# create headers for our new dataframe. These should correlate with the above.
df_input_headers = ["day", "temp", "dewp", "NUM_COLLISIONS"]
# create a final dataframe using our new dataframe and headers.
df_input = pd.concat(df_input_data, axis=1, keys=df_input_headers)

In [ ]:
# construct a training set for runnign through the model and a test set, we do this by using sample with 0.8 for an 80% training set and 20% for test.
training_set = df_input.sample(frac=0.8, random_state=0)
test_set = df_input.drop(training_set.index)

In [ ]:
# copy the datasets and remove the final column, i.e. the output column. We do this using pop.
training_features = training_set.copy()
test_features = test_set.copy()

training_labels = training_features.pop('NUM_COLLISIONS')
test_labels = test_features.pop('NUM_COLLISIONS')

In [ ]:
# Here I have put in a scale factor and divided by it. In this dataset, I had already normalised and thus it is 1. However, 10000 is what would make sense based on the data here and we can use this later when testing our model..
training_labels = training_labels/SCALE_NUM_COLLISIONS
test_labels = test_labels/SCALE_NUM_COLLISIONS

In [ ]:
# boiler plate for this model. You can see that we have used the training_features here for our normalisation layer that we try and fit to the outputs.
normaliser = tf.keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(training_features))

In [ ]:
# I have decided to call the model, model_1. We add our normaliser and we are expecting a single output.
model_1 = tf.keras.Sequential([
    normaliser,
    layers.Dense(units=1)
])

In [ ]:
# more boiler plate for creating a sequential model, we need an optimiser and loss parameter. Here we are going to be using the mean absolute error MAE
model_1.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
# now we are going to fit the model where we require the training features and labels. We will run it 100 times i.e. epochs and we have applied a further 20% validation split.

%%time
history = model_1.fit(
    training_features,
    training_labels,
    epochs=100,
    verbose=0,
    validation_split = 0.2)

CPU times: user 12.7 s, sys: 733 ms, total: 13.4 s
Wall time: 15.4 s


In [ ]:
# Now, we will evaluate our model using the test features and labels.
mean_absolute_error_model_1 = model_1.evaluate(
    test_features,
    test_labels, verbose=0)

In [ ]:
# The mean absolute error of the model can be printed out. Remember, we want to minimise this. Perhaps a model with just day and NUM_TRIPS would be better. It will also vary on each training run due to randomisation.
print(mean_absolute_error_model_1)

0.12655597925186157


In [ ]:
# we create a custom dataframe with 3 values per feature.
input_1 = pd.DataFrame.from_dict(data =
				{
            'day' : [1,1,1],
            'temp' : [83.6, 80.6, 79.7],
            'dewp' : [63, 65.5, 68.6]
        })

In [ ]:
input_1.head()

,day,temp,dewp
0,1,83.6,63.0
1,1,80.6,65.5
2,1,79.7,68.6


In [ ]:
# next we can check this out, you can multiply by 1000 to get more realistic NUM_COLLISIONS values.
linear_day_predictions_1 = model_1.predict(input_1[:3])*SCALE_NUM_COLLISIONS # essentially 1000 in this instance would give back realistic numbers based on the NUMBER OF COLLISIONS data
print(linear_day_predictions_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[[0.46710414]
 [0.45934793]
 [0.45915502]]


This test uses day 1 (Sunday) which shows lower number of collisions.

# Impact of Temperature and Dew Point:

As temperature decreases slightly and dew point increases, the model predicts a slight decrease in collision counts. This pattern suggests that the model has identified a weak positive relationship between temperature and collision counts, where higher temperatures may slightly increase the likelihood of collisions.
The small differences in predictions indicate that while temperature and dew point are affecting the predicted collision count, their impact is not very strong, resulting in only minor variations in the output.


These predictions imply that within the model, temperature and dew point have an influence on collision counts but a limited one. Since all values for day are constant, the model does not account for the weekly trend here, isolating the effect of weather conditions.
The predicted collision counts vary by a small margin (about 150 collisions across the three predictions), which aligns with previous observations that weather conditions (temperature and dew point) tend to have weak correlations with collision frequency.

The linear regression model predicts a slight decline in collision counts as temperature decreases and dew point increases. This suggests that, within the model's learned parameters, warmer and drier conditions may correlate with a marginally higher likelihood of collisions, but the effect size is relatively small. The consistent values for day indicate that day-of-week effects are controlled in this test, isolating the weather factors, which exhibit a weak influence on collision counts according to the model. This aligns with earlier correlation analyses showing that while weather factors contribute to collision predictions, they are not dominant predictors..

In [ ]:
# same as above
input_2 = pd.DataFrame.from_dict(data =
				{
            'day' : [7,7,7],
            'temp' : [81.9, 78.9, 67.2],
            'dewp' : [62.3, 60.7, 62.8]
        })

In [ ]:
linear_day_predictions_2 = model_1.predict(input_2[:3])*SCALE_NUM_COLLISIONS # essentially 1000 in this instance would give back realistic numbers based on the NUMBER OF COLLISIONS data
print(linear_day_predictions_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.8804865 ]
 [0.8690072 ]
 [0.83181125]]


This test uses day 7 (Friday) instead of day 1 (Sunday) which shows higher number of collisions. The other values were left the same.

# Impact of Temperature on Collisions:

As temperature decreases from 81.9 to 67.2, the model predicts a decrease in collision counts. This suggests that, on this specific day (Friday), higher temperatures are associated with a higher number of predicted collisions.
The drop in predicted collision counts as temperature falls reflects a pattern where warmer days might correlate with increased traffic or activities that lead to more collisions.

# Limited Influence of DewPoint:

Dew point remains relatively constant across the rows, and the model's predictions do not significantly shift due to dew point variations. This indicates that dew point has a limited impact on the number of collisions, at least for this day in the model’s learned parameters.


The model’s predictions indicate a slight but noticeable decrease in collision counts as temperature declines, suggesting that, on Fridays, higher temperatures correlate with a higher risk of collisions (from ~817 to ~757 collisions). Dew point variations have minimal influence. This aligns with the model’s broader pattern associating warmer conditions with increased collision counts, potentially due to greater travel or outdoor activity during warm weather. This trend highlights how specific days (like Fridays) may exhibit unique patterns in collision risk, influenced modestly by weather conditions.